In [1]:
# Imports
import os
from tqdm import tqdm
import cv2
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import layers
from keras.models import Model
from tensorflow.keras.utils import Sequence
from keras.optimizers import Adam
from keras import backend as K
from keras.models import load_model

from helper_functions import get_picture_filenames_from_folder, load_and_transform_pictures, display_blur_sharp_and_pred_images

2023-12-23 00:52:38.964052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-23 00:52:38.964208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-23 00:52:39.174545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-23 00:52:39.320765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-23 00:52:49.936208: W tensorflow/compiler/tf2

In [2]:
# Setup
plt.rcParams["figure.figsize"] = [9, 16]

In [3]:
# Check if GPU is available
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Path helpers
blurred_train_path = os.path.join('clean_data', 'train', 'blur')
sharp_train_path = os.path.join('clean_data', 'train', 'sharp')
blurred_test_path = os.path.join('clean_data', 'test', 'blur')
sharp_test_path = os.path.join('clean_data', 'test', 'sharp')

In [5]:
# Data generator class
class DataGenerator(Sequence):
    # Based on https://github.com/AryanSethi/No-Blur/blob/master/model_exps/try4.ipynb
    
    def __init__(self, base_dir,base_dir2, output_size, shuffle=False, batch_size=10):
        self.base_dir = base_dir
        self.base_dir2 = base_dir2
        self.output_size = output_size
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.all_x = [f for f in os.listdir(base_dir) if f.endswith(".png")]
        self.all_y = [f for f in os.listdir(base_dir2) if f.endswith(".png")]
        self.on_epoch_end()

    def on_epoch_end(self):
        self.indices = np.arange(len(self.all_x))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return int(len(self.all_x) / self.batch_size)

    def __getitem__(self, idx):
        X = np.empty((self.batch_size, *self.output_size, 3))
        Y = np.empty((self.batch_size, *self.output_size, 3))

        indices = self.indices[idx*(self.batch_size): (idx+1)*(self.batch_size)]

        for i,j in enumerate(indices):
            img_path = os.path.join(self.base_dir,self.all_x[j])
            img_path2 = os.path.join(self.base_dir2,self.all_y[j])

            img  = cv2.imread(img_path)
            img2 = cv2.imread(img_path2)
            X[i,] = img
            Y[i,] = img2
        X= X.astype('float32')/255
        Y= Y.astype('float32')/255
        return X, Y

In [6]:
# This autoencoder structure is based on https://github.com/AryanSethi/No-Blur/blob/master/model_exps/try4.ipynb

def conv_operation(x, filters, kernel_size, strides=2):
    x = layers.Conv2D(filters=filters,
              kernel_size=kernel_size,
              strides=strides,
              padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def conv_transpose_operation(x, filters, kernel_size):
    x = layers.Conv2DTranspose(filters=filters,
                       kernel_size=kernel_size,
                       strides=2,
                       padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def deblurring_autoencoder():
    dae_inputs = layers.Input(shape=(720, 1280, 3), name='dae_input')
    conv_block1 = conv_operation(dae_inputs, 64, 3)
    conv_block2 = conv_operation(conv_block1, 128, 3)
    conv_block3 = conv_operation(conv_block2, 256, 3)
    conv_block4 = conv_operation(conv_block3, 512, 3)

    conv_block5 = conv_operation(conv_block4, 512, 3, 1)

    deconv_block1 = conv_transpose_operation(conv_block5, 512,3)
    merge1 = layers.Concatenate()([conv_block3,deconv_block1])
    deconv_block2 = conv_transpose_operation(merge1, 256, 3)
    merge2 = layers.Concatenate()([deconv_block2, conv_block2])
    deconv_block3 = conv_transpose_operation(merge2, 128, 3)
    merge3 = layers.Concatenate()([deconv_block3, conv_block1])
    deconv_block4 = conv_transpose_operation(merge3, 64, 3)

    final_deconv = layers.Conv2DTranspose(filters=3, kernel_size=3,padding='same')(deconv_block4)
    return Model(dae_inputs, final_deconv, name='dae')

In [7]:
model = deblurring_autoencoder()
model.summary()

2023-12-23 00:53:05.062801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30953 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:09:00.0, compute capability: 7.0


Model: "dae"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 dae_input (InputLayer)      [(None, 720, 1280, 3)]       0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 360, 640, 64)         1792      ['dae_input[0][0]']           
                                                                                                  
 batch_normalization (Batch  (None, 360, 640, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 360, 640, 64)         0         ['batch_normalization[0][0]'

In [8]:
# Define model inputs, compile model

train = DataGenerator(blurred_train_path, sharp_train_path, (720,1280), batch_size=8, shuffle=False)
test  = DataGenerator(blurred_test_path, sharp_test_path, (720,1280), batch_size=8, shuffle=False)
opt=Adam(learning_rate=0.001)

now = datetime.now()
checkpoint_filepath = f"models/autoencoder_{now.year}_{now.month}_{now.day}T{now.hour}_{now.minute}_epoch"+"{epoch:02d}_mae_{loss:.4f}_val_mae_{val_loss:.4f}.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose = 1,
    save_best_only=False)

model.compile(optimizer=opt, loss='mae')

In [9]:
# Load model to continue training
model_name = 'autoencoder_2023_12_22T18_48_epoch55_mae_0.0349_val_mae_0.0333.keras'
model_path = 'models'
model = load_model(os.path.join(model_path, model_name))

In [ ]:
# Model training

history = model.fit(
    train,
    epochs=100,
    callbacks = [model_checkpoint_callback],
    validation_data = test
)

Epoch 1/100


2023-12-23 00:53:34.328353: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-23 00:54:02.430352: I external/local_xla/xla/service/service.cc:168] XLA service 0x2ace2071eac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-23 00:54:02.430431: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-12-23 00:54:02.446915: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703285642.760206   22139 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


262/262 [==============================] - ETA: 0s - loss: 0.0349
Epoch 1: saving model to models/autoencoder_2023_12_23T0_53_epoch01_mae_0.0349_val_mae_0.0369.keras
262/262 [==============================] - 544s 2s/step - loss: 0.0349 - val_loss: 0.0369
Epoch 2/100
262/262 [==============================] - ETA: 0s - loss: 0.0350
Epoch 2: saving model to models/autoencoder_2023_12_23T0_53_epoch02_mae_0.0350_val_mae_0.0363.keras
262/262 [==============================] - 544s 2s/step - loss: 0.0350 - val_loss: 0.0363
Epoch 3/100
262/262 [==============================] - ETA: 0s - loss: 0.0344
Epoch 3: saving model to models/autoencoder_2023_12_23T0_53_epoch03_mae_0.0344_val_mae_0.0332.keras
262/262 [==============================] - 534s 2s/step - loss: 0.0344 - val_loss: 0.0332
Epoch 4/100
262/262 [==============================] - ETA: 0s - loss: 0.0345
Epoch 4: saving model to models/autoencoder_2023_12_23T0_53_epoch04_mae_0.0345_val_mae_0.0332.keras
262/262 [=====================

In [ ]:
# Load best model
model_name = 'autoencoder_2023_12_22T18_48_epoch55_mae_0.0349_val_mae_0.0333.keras'
model_path = 'models'
best_model = load_model(os.path.join(model_path, model_name))

In [ ]:
# Load in data splits for testing
num_of_test_files = 10
test_batch_filenames = get_picture_filenames_from_folder(blurred_test_path)[:num_of_test_files]
blurred_test_batch = load_and_transform_pictures(test_batch_filenames, blurred_test_path)
sharp_test_batch = load_and_transform_pictures(test_batch_filenames, sharp_test_path)

In [ ]:
# Predicting with best model
predictions = np.clip(best_model.predict(blurred_test_batch), 0, 1).astype('float64')

In [ ]:
# Display blurred, predicted and sharp images
img_id = 8
display_blur_sharp_and_pred_images(img_id, blurred_test_batch, sharp_test_batch, predictions)